# Make labels

The labels from the highest quality assignment for each site were rasterized into 3 classes: non-field (0), field interior (1), and field edge, which provides the basis for training a boundary aware semantic segmentation model that can be used to more effectively separate individual fields.

The resulting chips provide a single label for each image chip. Additional assignments for Class 1 and 4 sites are also available, and can be converted to chips for different purposes (e.g. to assess the impact of label noise on model predictions; Elmes et al, 2020).

This notebook may also be adapted to create different types of labels, such as binary labels (field/no-field) or separate labels representing, for example, field edge, field interior, non-field, and distance to nearest field boundary (cite).

In [ ]:
# from google.colab import drive
# drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [5]:
%%capture
!pip install rasterio
!pip install rioxarray
!pip install pandas==2.2.1
!pip install geopandas
!pip install leafmap
!pip install pyarrow

In [6]:
import os
import re
import warnings
from pathlib import Path
import rioxarray as rxr
import xarray as xr
from rasterio.enums import Resampling
from rasterio import features
import geopandas as gpd
from shapely.geometry import Polygon, box, mapping
import pandas as pd
import numpy as np
from datetime import datetime as dt
import multiprocessing as mp
import leafmap.leafmap as leafmap
from pyarrow import parquet as parq

## Set-up

In [7]:
root_dir = "/home/airg/fharrington"
proj_dir = "/home/airg/fharrington/fieldmapperfh"
# data_dir = Path(proj_dir) / "data"
chip_dir = "/home/airg/lestes/data/lacuna/images"
label_dir = "/home/airg/fharrington/fieldmapperfh/labels"
final_dir = "/home/airg/fharrington/fieldmapperfh/processed"
# if not os.path.exists(label_dir):
#     os.makedirs(label_dir, exist_ok=True)
log_path = "/home/airg/fharrington/fieldmapperfh/logs/label-maker2.log"

In [8]:
label_dir

'/home/airg/fharrington/fieldmapperfh/labels'

### Functions

## Selecting a label catalog

The label catalog provided here provides all labels collected during the project, with the exception of sites where image quality was insufficient to label (note: additional labels may have been retained with low image quality).  Labels are provided in the following classes:

The labels have several distinct classes:

-   Class 1a: A subset expert-labeled sites that were selected to serve as quality control sites (Q sites) in the labelling platform;

-   Class 1b: Expert-labeled sites not used in the labelling platform for Quality control;

-   Class 1c: Q control assignments completed by the labelling teams against Class 1a labels;

-   Class 1d: Sites corresponding to Class 1b sites that were digitized by 1-3 members of the labeling team;

-   Class 2: Ordinary mapping assignments, represented a single unique site mapped by a single labeller. An exception to this case of one labeller/one label were for those assignment marked as Untrusted, in which case it would have been mapped multiple times until the first approved assignment was completed.

-   Class 4: Sites mapped by three separate labellers

The catalog provides a range of quality metrics associated with the labels, with the following exceptions;

- For Class 1a and 1b, no Qscores are available
- For Class 1b, there are 153 sites for which no Rscore is available

All labels are preserved in the catalog, as they may be useful for different purposes, such as:

- Contructing quality-weighted consensus maps to:
    1. Assess label uncertainty;
    2. Extract pixel-based samples from area of higher certainty
- Testing whether additional noisy labels added to training samples improve model performance;
- Pass multiple labels for the same location into a model as a form of data-based regularization

We provided an example here of one approach for selecting what is likely to be the best label for each site, using the following rules:

- Drop:
    - All assignments marked as Untrusted or Rejected;
    - No Class 1c labels;
- Keep:
    - All Class 1a assignments;
    - All Class 2 assignments;
    - The assignment with the highest Rscore for each site occuring in one or both of the Class 1b and Class 1d samples;
    - The assignment with the highest Rscore for each Class 4 site

Alternative approaches may also be used for finding the best sample, such as using the highest Qscore per sample, or multiple individual Q metrics, e.g. the highest combination of Area and N metrics. It may also be preferrable to take the Class 1c assignment with the highest Qscore or highest Rscore for each Class 1a site, in place of the Class 1a labels.

In [9]:
def filter_catalog(catalog, groups, metric, keep):
    """
    Function to filter the full catalog by class and quality metric

    Args:
    catalog: DataFrame
        The full catalog, read in
    groups: List
        A list of key-value pairs providing with possible keys of "whole" and
        "best", with the values providing one or more of the label classes.
        Classes corresponding to "whole" will have all assignments in the class
        selected. "Best" will result in the best assignment
        corresponding to the provided metric selected, "worst" will return the
        lowest assignment for the metric.
    metric: str
        One of the quality metrics in the catalog, e.g. Rscore, Qscore. Must be
        provided if a key in groups is not "whole"
    keep: list
        Names of columns in the full catalog that should be kept in the
        filtered catalog

    Returns:
        A DataFrame containing the filtered assignments, possibly with
        duplicates. If so, you may wish to remove them by following up with a
        `drop_duplicates()`
    """
    out_catalog = []
    for g in groups:
        cls = list(g.values())[0]
        cat = catalog.query("Class in @cls")
        if list(g.keys())[0] == "whole":
            print(f"Extracting all of Class {' and '.join(cls)}")
            out_catalog.append(cat)
        elif list(g.keys())[0] == "best":
            print(f"Extracting best of Class {' and '.join(cls)}")
            out_catalog.append(
                cat.groupby("name")
                .apply(lambda x: x.loc[[x[metric].idxmax()]]
                       if not x[metric].isna().all() else x,
                       include_groups=False)
                .reset_index(level=["name"])
            )
        elif list(g.keys())[0] == "worst":
            print(f"Extracting worst of Class {' and '.join(cls)}")
            out_catalog.append(
                cat.groupby("name")
                .apply(lambda x: x.loc[[x[metric].idxmin()]]
                       if not x[metric].isna().all() else x,
                       include_groups=False)
                .reset_index(level=["name"])
            )
        else:
            print("Use either 'whole' or 'best' as group keys")
            break

    out_catalog = pd.concat(out_catalog, axis=0)[keep].reset_index(drop=True)
    return out_catalog


def threeclass_label(assignment, label_dir, fields, log=None,
                     overwrite=True):
    """
    Create a three class label (0: non-field, 1: field interior,
    2: field boundary) with the same dimensions as the corresponding
    image chip.

    Args:
    assignment: pandas.Series
        A series representing one assignment from the label catalog
    fields: geopandas.GeoDataFrame
        The fields polygons, read in from the provided geoparquet file
    label_dir: str or Path
        Directory to write rasterized labels to
    log: str or Path
        Name and path of log file
    overwrite: bool
        Overwrite label if it exists on disk or not (default = True)
    """

    lbl_name = f"{re.sub('.tif', '', assignment.chip)}-"\
        f"{assignment.assignment_id}.tif"
    dst_path = Path(label_dir) / lbl_name

    if not overwrite and os.path.exists(dst_path):
       msg = f"{os.path.basename(dst_path)} exists, skipping"
       print(msg, file=log, flush=True)

    else:
        chip = rxr.open_rasterio(Path(chip_dir) / assignment.chip)

        transform = chip.rio.transform()
        _, r, c = chip.shape
        res = np.mean([abs(transform[0]), abs(transform[4])])

        grid = gpd.GeoDataFrame(geometry=[box(*chip.rio.bounds())],
                                crs=chip.rio.crs)

        out_arr = np.zeros((r, c)).astype('int16')
        if assignment.nflds > 0:

            shp = fields.query("assignment_id==@row.assignment_id").copy()

            shp["category"] = 1
            shp['buffer_in'] = shp.geometry.buffer(-res)
            shp['buffer_out'] = shp.geometry.buffer(res)
            shp = gpd.overlay(grid, shp, how='intersection')
            out_arr = np.zeros((r, c)).astype('uint8')

            shapes = ((geom, value)
                      for geom, value in zip(shp['geometry'], shp['category']))
            burned = features.rasterize(shapes=shapes, fill=0,
                                        out=out_arr.copy(),
                                        transform=transform)

            try:
                shapes_shrink = (
                    (geom, value)
                    for geom, value in zip(shp['buffer_in'], shp['category'])
                )
                shrunk = features.rasterize(
                    shapes=shapes_shrink, fill=0, out=out_arr.copy(),
                    transform=transform
                )
                shapes_explode = (
                    (geom, value)
                    for geom, value in zip(shp['buffer_out'], shp['category'])
                )
                exploded = features.rasterize(
                    shapes=shapes_explode, fill=0, out=out_arr.copy(),
                    transform=transform
                )
            except:
                shp['buffer'] = shp.geometry.buffer(-res)
                shapes_shrink = (
                    (geom, value)
                    for geom, value in zip(shp['buffer'], shp['category'])
                )
                shrunk = features.rasterize(
                    shapes=shapes_shrink, fill=0, out=out_arr.copy(),
                    transform=transform
                )

            lbl = (
                burned * 2 - shrunk + \
                np.where((exploded*2-burned)==1, 0, exploded*2-burned)
                .astype(np.uint8)
            )
        else:
            lbl = out_arr

        lbl_raster = xr.DataArray(
            lbl,
            dims=["y", "x"],
            coords={"y": chip.y, "x": chip.x},
            attrs={"transform": transform, "crs": chip.rio.crs}
        )

        # check dimensions
        try:
            assert chip.rio.bounds() == lbl_raster.rio.bounds()
        except AssertionError as err:
            msg = f"{dt.now()}: {os.path.basename(dst_path)} has incorrect bounds"
            print(msg, file=log, flush=True)
            raise err
        try:
            assert chip.shape[1:3] == lbl_raster.shape
        except AssertionError as err:
            msg = f"{dt.now()}: {os.path.basename(dst_path)} incorrect output shape"
            print(msg, file=log, flush=True)
            raise err

        # write to disk
        lbl_raster.rio.to_raster(dst_path)
        msg = f"{dt.now()}: created {os.path.basename(dst_path)}"
        print(msg, file=log, flush=True)

    assignment_out = assignment.copy()
    assignment_out["label"] = lbl_name

    return pd.DataFrame([assignment_out.to_list()],
                        columns=assignment_out.index)

def view_random_label(label_catalog, label_dir, chip_dir, bands=[1,2,3],
                      seed=None):
    """
    A leafmap-based viewer that enables comparison of a randomly selected
    label against of the image chip

    Args:
    label_catalog: pandas.DataFrame
        The processed label catalog
    label_dir: str
        The path to the label directory (not strings only, not Path)
    chip_dir: str
        The path to the image chip directory (not strings only, not Path)
    bands: list
        Specify band combination for the image chip. Defaults to [1,2,3] for
        true color
    seed: int
        Defaults to None, but the same chip can be selected again if an integer
        is provided

    Returns:
        A leafmap viewer
    """
    random_label = label_catalog.sample(n=1, random_state=seed)
    lbl_path = os.path.join(label_dir, random_label.label.iloc[0])
    chip_path = os.path.join(chip_dir, random_label.chip.iloc[0])
    lbl_name = re.sub(".tif", "", random_label.label.iloc[0])
    m = leafmap.Map(
        zoom=17, center=random_label[["y", "x"]].iloc[0].to_list()
    )
    m.add_basemap("SATELLITE")
    m.add_raster(chip_path, bands=[1,2,3], layer_name='Image',
                 zoom_to_layer=False)
    m.add_raster(lbl_path, layer_name=lbl_name,zoom_to_layer=False)
    # m.drop_duplicates()
    return m

### Read in catalog

In [10]:
catalog = pd.read_csv("/home/airg/fharrington/fieldmapperfh/label_catalog_allclasses.csv",
                      low_memory=False)

In [ ]:
len(catalog.query("status == 'None'"))

0

In [11]:
'''
Create catalog selections as dataframes to pass to filter_catalog()
'''

# drop labels without an "rscore" (lowercase r)
hasrscore = catalog.query("rscore != 'NA'")
# create set with rscore >= 2
best = hasrscore.query("rscore >= 2")
# drop duplicate names (geog location) from best set
bestunique  = best.drop_duplicates("name")

# drop names NOT in best set from remaining catalog
remain = catalog[~catalog["name"].isin(best["name"])]

# finding num of values to complete 30% pool for test/train
numremainbest_toselect = (
    int(((len(pd.unique(catalog["name"]))*0.3-len(bestunique))))
)
# select remainder of validate + test set from those with best Rscores
# (uppercase R)
remain_best = (
    remain
    .sort_values(by='Rscore')
    .head(numremainbest_toselect)
    )
# create best set from remain_best
bestset = pd.concat([best, remain_best], ignore_index=True)
len(bestset)

#make remaining catalog with names NOT in bestset catalog
#THIS IS WHAT YOU RUN WITH filter_labels(catalog_no_best, ..., ..., min/max)
catalog_no_best = catalog[~catalog["name"].isin(bestset["name"])]
catalog_no_best.insert(2, "usage", "train")

# create validation set sample 2/3 then drop and the remaining 3rd is test
val_set = bestset.sample(int(len(bestset)*(2/3)), random_state = 1)
val_set.insert(2, "usage", "validate")

# create test set (drop val set from bestset)
test_set = bestset.drop(val_set.index)
test_set.insert(2, "usage", "test")

#regroup val/test set this time with labels
val_test_set = pd.concat([val_set, test_set], ignore_index=True)

### Apply filter

To follow the selection example describe above, we first queried the catalog to drop Untrusted and Rejected assignments, then defined a grouping list with the key for "whole" Classes to be retained set at "1a" and "2", and two separate groups for "best" selection. The first is for Classes "1b" and "1d", which means that if there are multiple assignments for the same site in those classes, the one with the highest metric (here Rscore) was selected. The second is for Class 4, and the same selection process was followed within that Class, picking the single assignment with the highest Rscore.  

Since some sites were mapped in Classes 1a and 1b, a final filtering is applied in which the Class 1a assignments were retained in preference to assignments for the same sites in Class 1b.

In [23]:
catalog_no_best.head()

,name,Class,usage,assignment_id,Labeller,completion_time,label_time,status,Score,N,...,Qscore,QN,Rscore,x,y,farea,nflds,tile,image_date,chip
0,ET0007182,2,train,50,22,2023-11-13T19:09:18Z,1.416667,Untrusted,NaN,NaN,...,0.621788,0.361483,0.706386,39.5115,14.4825,0.000000,0,430022.0,2017-08-15,ET0007182-2017-08-15.tif
1,NE3372442,2,train,51,22,2023-11-13T19:10:12Z,0.900000,Untrusted,NaN,NaN,...,0.621788,0.361483,0.706386,9.1515,14.4025,0.000000,0,417654.0,2021-08-15,NE3372442-2021-08-15.tif
3,SD4068077,2,train,52,22,2023-11-13T19:12:49Z,2.616667,Untrusted,NaN,NaN,...,0.621788,0.361483,0.706386,33.3765,14.3925,6.902422,7,427259.0,2022-03-15,SD4068077-2022-03-15.tif
22,SN0220055,2,train,30,22,2023-11-10T11:27:30Z,38.400000,Untrusted,NaN,NaN,...,0.621788,0.361483,0.706386,-16.8085,14.9225,1.528770,15,383855.0,2019-02-15,SN0220055-2019-02-15.tif
24,SD3855777,2,train,32,22,2023-11-10T11:39:34Z,7.616667,Untrusted,NaN,NaN,...,0.621788,0.361483,0.706386,35.0465,14.7725,0.000000,0,404712.0,2020-02-15,SD3855777-2020-02-15.tif


In [30]:
# catalog = catalog.query("status not in ['Untrusted', 'Rejected']")
# Create label catalogs for worst and best to concat with val/test and pass to
# label chip maker
groupsbest = [
    {"whole": ["1a", "2"]},
    {"best": ["1b", "1d"]},
    {"best": "4"}
]

groupsworst = [
    {"whole": ["1a", "2"]},
    {"worst": ["1b", "1d"]},
    {"worst": "4"}
]

keep = ["name", "usage", "Class", "assignment_id", "Labeller", "status", "Score",
        "N", "Area", "Qscore", "Rscore", "x", "y", "farea", "nflds",
       "chip"]

train_worst = filter_catalog(catalog_no_best, groupsworst, "Rscore", keep)
train_best = filter_catalog(catalog_no_best, groupsbest, "Rscore", keep)
# there are some duplicates of Class 1a in Classes 1b/d, so we'll drop those
train_worst.drop_duplicates("name", inplace=True)
train_best.drop_duplicates("name", inplace=True)


Extracting all of Class 1a and 2
Extracting worst of Class 1b and 1d
Extracting worst of Class 4
Extracting all of Class 1a and 2
Extracting best of Class 1b and 1d
Extracting best of Class 4


In [49]:
train_best.head()

,name,usage,Class,assignment_id,Labeller,status,Score,N,Area,Qscore,Rscore,x,y,farea,nflds,chip
0,ET0007182,train,2,50,22,Untrusted,NaN,NaN,NaN,0.621788,0.706386,39.5115,14.4825,0.000000,0,ET0007182-2017-08-15.tif
1,NE3372442,train,2,51,22,Untrusted,NaN,NaN,NaN,0.621788,0.706386,9.1515,14.4025,0.000000,0,NE3372442-2021-08-15.tif
2,SD4068077,train,2,52,22,Untrusted,NaN,NaN,NaN,0.621788,0.706386,33.3765,14.3925,6.902422,7,SD4068077-2022-03-15.tif
3,SN0220055,train,2,30,22,Untrusted,NaN,NaN,NaN,0.621788,0.706386,-16.8085,14.9225,1.528770,15,SN0220055-2019-02-15.tif
4,SD3855777,train,2,32,22,Untrusted,NaN,NaN,NaN,0.621788,0.706386,35.0465,14.7725,0.000000,0,SD3855777-2020-02-15.tif


In [26]:
train_worst.describe()

,assignment_id,Score,N,Area,Qscore,Rscore,x,y,farea,nflds
count,24024.000000,0.0,0.0,0.0,23417.000000,24013.000000,24024.000000,24024.000000,24024.000000,24024.000000
mean,21886.695929,NaN,NaN,NaN,0.612915,0.745695,20.860349,2.377689,1.644232,21.251374
std,17026.067214,NaN,NaN,NaN,0.020518,0.071922,15.499730,12.238922,4.386972,19.996433
min,30.000000,NaN,NaN,NaN,0.578685,0.630252,-17.138500,-29.977500,0.000000,0.000000
25%,9501.750000,NaN,NaN,NaN,0.592077,0.691748,8.000250,-5.457500,0.437026,7.000000
50%,20031.500000,NaN,NaN,NaN,0.619660,0.728205,27.319000,8.547500,0.797031,16.000000
75%,30994.250000,NaN,NaN,NaN,0.625471,0.811741,33.896500,11.853750,1.474738,30.000000
max,102190.000000,NaN,NaN,NaN,0.653840,0.870748,47.256500,18.217500,197.492395,299.000000


The final class counts were as follows

In [28]:
# train_worst.value_counts("Class")
train_best.value_counts("Class")

Class
2     22652
1d      479
4       459
1a      262
1b      172
Name: count, dtype: int64

In [59]:
#join train sets and val/test set for making label chips
labelset_worst = pd.concat([train_worst, val_test_set], ignore_index=True)
labelset_best = pd.concat([train_best, val_test_set], ignore_index=True)

In [60]:
train_worst.head()

,name,usage,Class,assignment_id,Labeller,status,Score,N,Area,Qscore,Rscore,x,y,farea,nflds,chip
0,ET0007182,train,2,50,22,Untrusted,NaN,NaN,NaN,0.621788,0.706386,39.5115,14.4825,0.000000,0,ET0007182-2017-08-15.tif
1,NE3372442,train,2,51,22,Untrusted,NaN,NaN,NaN,0.621788,0.706386,9.1515,14.4025,0.000000,0,NE3372442-2021-08-15.tif
2,SD4068077,train,2,52,22,Untrusted,NaN,NaN,NaN,0.621788,0.706386,33.3765,14.3925,6.902422,7,SD4068077-2022-03-15.tif
3,SN0220055,train,2,30,22,Untrusted,NaN,NaN,NaN,0.621788,0.706386,-16.8085,14.9225,1.528770,15,SN0220055-2019-02-15.tif
4,SD3855777,train,2,32,22,Untrusted,NaN,NaN,NaN,0.621788,0.706386,35.0465,14.7725,0.000000,0,SD3855777-2020-02-15.tif


## Making label chips

The next step was to convert the polygons into chips. This requires the geoparquet file containing the field polygons. We will use the filtered label catalog to get the image chip that corresponds to the labelled site, and the convert the polygons for each site to a 3-class label that has the same dimensions as the image chip.

In [32]:
fields = gpd.read_parquet("/home/airg/fharrington/fieldmapperfh/mapped_fields_final.parquet")


In [33]:
#set labeltype based on quality
def set_label_type(quality):
    if quality == "best":
        labelset = labelset_best
    elif quality == "worst":
        labelset = labelset_worst        
    else:
        labelset = ""
        print('function argument should be either "best" or "worst"')
    return labelset


In [34]:
#set labeldir based on quality
def set_label_dir(quality):
    if quality == "best":
        labeldir = "/home/airg/fharrington/fieldmapperfh/labels_best"
    elif quality == "worst":
        labeldir = "/home/airg/fharrington/fieldmapperfh/labels_worst"       
    else:
        print('please provide an argument of "best" or "worst" to set the label directory')
    return labeldir

In [67]:
#choose label catalog to use for label creation
quality = "best"
label_set = set_label_type(quality)
label_dir = set_label_dir(quality)

In [75]:
label_set.head()

,name,usage,Class,assignment_id,Labeller,status,Score,N,Area,Qscore,...,label_time,Edge,FieldSkill,NoFieldSkill,Categorical,rscore,rscore2,QN,tile,image_date
0,ET0007182,train,2,50,22,Untrusted,NaN,NaN,NaN,0.621788,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NE3372442,train,2,51,22,Untrusted,NaN,NaN,NaN,0.621788,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SD4068077,train,2,52,22,Untrusted,NaN,NaN,NaN,0.621788,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SN0220055,train,2,30,22,Untrusted,NaN,NaN,NaN,0.621788,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SD3855777,train,2,32,22,Untrusted,NaN,NaN,NaN,0.621788,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
warnings.filterwarnings(
    'ignore',
    message="Geometry is in a geographic CRS. Results from " +\
    "'buffer' are likely incorrect."
)
log = open(log_path, "a+")
print(f"\nStarting at {dt.now()}\n", file=log, flush=True)
lbls = []

for i, row in label_set.iterrows():
    lbls.append(threeclass_label(row, label_dir=label_dir, fields=fields,
                                 log=log, overwrite=False))

print(f"\nFinished at {dt.now()}", file=log, flush=True)
log.close()

In [70]:
lbls[10]

,name,usage,Class,assignment_id,Labeller,status,Score,N,Area,Qscore,...,Edge,FieldSkill,NoFieldSkill,Categorical,rscore,rscore2,QN,tile,image_date,label
0,SD3943818,train,2,49,22,Untrusted,NaN,NaN,NaN,0.621788,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SD3943818-2018-08-15-49.tif


### Save catalog

In [73]:
label_catalog_final = pd.concat(lbls).reset_index(drop=True)
label_catalog_final.to_csv("/home/airg/fharrington/fieldmapperfh/label-catalog-filtered_" + quality + ".csv")

## Display labels

The following viewer can be used to display labels against the images

In [34]:
view_random_label(label_catalog_final, label_dir, chip_dir)

ImportError: localtileserver is not installed. Please install it before proceeding. https://github.com/banesullivan/localtileserver

In [ ]:
fields.shape

(825395, 6)